In [ ]:
import os
from utils import *
import utils
from agents import *
import time
import torch
import torch.nn as nn
from copy import deepcopy
import argparse

In [10]:
parser = argparse.ArgumentParser()
parser.add_argument('--seeds',          type=int,       default=[2023, 2024, 2025])
parser.add_argument('--dataset',        type=str,       default='svhn')
parser.add_argument('--batch_size',     type=int,       default=512)
parser.add_argument('--model_name',     type=str,       default='vgg11')
parser.add_argument('--retrain',        type=bool,      default=False)
parser.add_argument('--unlearn_class',  type=list,      default=3)
args = parser.parse_args("")
args.time_str = time.strftime("%m-%d-%H-%M", time.localtime())
if args.dataset.lower() == 'fmnist':
    args.n_channels = 1
else:
    args.n_channels = 3

if args.dataset.lower() == 'cifar100':
    args.num_classes = 100
else:
    args.num_classes = 10
criterion = nn.CrossEntropyLoss()

In [11]:
dict = {
    1: {'unlean_class': [3], 'arxiv_name': '12-19-23-47'},
    2: {'unlean_class': [1, 3], 'arxiv_name': '12-27-22-37'},
    3: {'unlean_class': [1, 3, 5], 'arxiv_name': '12-28-11-53'},
    4: {'unlean_class': [1, 5, 8, 3], 'arxiv_name': '12-27-20-22'},
    5: {'unlean_class': [1, 5, 6, 7, 9], 'arxiv_name': '12-28-00-01'},
}
num_unlearn = 1
args.unlean_class = dict[num_unlearn]['unlean_class']
arxiv_name = dict[num_unlearn]['arxiv_name']

arxiv_name_o = '12-19-21-49'

In [12]:
train_loader, val_loader, test_loader = get_dataloader(args)

Using downloaded and verified file: ../data/svhn/train_32x32.mat
Using downloaded and verified file: ../data/svhn/test_32x32.mat


# Unlearn 1 classes

In [9]:
num_unlearn = 1
arxiv_name = dict[num_unlearn]['arxiv_name']
args.unlearn_class = dict[num_unlearn]['unlean_class']
Acc_r, Acc_f = np.zeros((2,3)), np.zeros((2,3))
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    Acc_f[0][i], Acc_r[0][i] = test_by_class(model, test_loader, i=args.unlearn_class)

print('------------ Retrained model ------------')
for i in range(3):
    model_r = get_model(args)
    try:
        model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/retrain_model_{arxiv_name}_{args.seeds[i]}.pth'))
        Acc_f[1][i], Acc_r[1][i] = test_by_class(model_r, test_loader, i=args.unlearn_class)
    except:
        print('No retrained model')
        break

print(f'Original model Acc_f: {100*Acc_f[0].mean():.2f} \pm {100*Acc_f[0].std():.2f}')
print(f'Original model Acc_r: {100*Acc_r[0].mean():.2f} \pm {100*Acc_r[0].std():.2f}')

print(f'Retrained model Acc_f: {100*Acc_f[1].mean():.2f} \pm {100*Acc_f[1].std():.2f}')
print(f'Retrained model Acc_r: {100*Acc_r[1].mean():.2f} \pm {100*Acc_r[1].std():.2f}')


0.9719, 0.9712, 0.9624, [0.9167], 0.9687, 0.9488, 0.9474, 0.9534, 0.9253, 0.9611, Acc_f: 0.9167, Acc_r: 0.9567
0.9639, 0.9722, 0.9658, [0.9129], 0.9691, 0.9480, 0.9646, 0.9287, 0.9253, 0.9473, Acc_f: 0.9129, Acc_r: 0.9539
0.9616, 0.9645, 0.9655, [0.9094], 0.9734, 0.9534, 0.9560, 0.9376, 0.9361, 0.9461, Acc_f: 0.9094, Acc_r: 0.9549
------------ Retrained model ------------
0.9696, 0.9651, 0.9638, [0.0000], 0.9774, 0.9602, 0.9469, 0.9257, 0.9247, 0.9386, Acc_f: 0.0000, Acc_r: 0.9524
0.9713, 0.9672, 0.9737, [0.0000], 0.9675, 0.9576, 0.9525, 0.9406, 0.9307, 0.9586, Acc_f: 0.0000, Acc_r: 0.9578
0.9553, 0.9649, 0.9689, [0.0000], 0.9679, 0.9610, 0.9509, 0.9450, 0.9380, 0.9580, Acc_f: 0.0000, Acc_r: 0.9567
Original model Acc_f: 91.30 \pm 0.30
Original model Acc_r: 95.52 \pm 0.12
Retrained model Acc_f: 0.00 \pm 0.00
Retrained model Acc_r: 95.56 \pm 0.23


In [ ]:
train_loader, val_loader, test_loader = get_dataloader(args)
remain_train_loader, unlearn_train_loader = split_2_remain_unlearn(args, train_loader)
remain_val_loader, unlearn_val_loader = split_2_remain_unlearn(args, val_loader)
remain_test_loader, unlearn_test_loader = split_2_remain_unlearn(args, test_loader)
remain_class = np.setdiff1d(np.arange(args.num_classes), args.unlearn_class)

Proj_mat_lst =[]
train_targets_list = np.array(train_loader.dataset.labels)[train_loader.sampler.indices]
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    print(f'------------ Trail {i} ------------')
    merged_feat_mat = []
    for cls_id in range(10): 
        cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
        cls_indices = train_loader.sampler.indices[cls_indices]
        cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
        cls_loader_dict = torch.utils.data.DataLoader(train_loader.dataset, 
                                                                batch_size=args.batch_size, 
                                                                sampler=cls_sampler)
        if cls_id in args.unlearn_class:
            continue
        for batch, (x, y) in enumerate(cls_loader_dict ):
            x = x.cuda()
            y = y.cuda()
            mat_list = get_representation_matrix(model, 
                                                x, 
                                                batch_list=[24, 100, 100, 100, 125, 125, 125, 250, 250, 256, 256])
            break
        threshold = 0.97 + 0.003*cls_id
        merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
        proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]
        Proj_mat_lst.append(proj_mat)

In [26]:
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.0005)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(6):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = torch.from_numpy(np.random.choice(remain_class, size=x.shape[0])).cuda()
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9719, 0.9712, 0.9624, [0.9167], 0.9687, 0.9488, 0.9474, 0.9534, 0.9253, 0.9611, Acc_f: 0.9167, Acc_r: 0.9567
[train] epoch 0, batch 14, loss 3.767608404159546
0.9736, 0.9743, 0.9600, [0.6721], 0.9719, 0.9564, 0.9464, 0.9287, 0.9108, 0.9611, Acc_f: 0.6721, Acc_r: 0.9537
[train] epoch 1, batch 14, loss 2.866029977798462
0.9759, 0.9751, 0.9489, [0.2318], 0.9742, 0.9543, 0.9393, 0.9143, 0.8837, 0.9561, Acc_f: 0.2318, Acc_r: 0.9469
[train] epoch 2, batch 14, loss 2.621795415878296
0.9765, 0.9751, 0.9426, [0.1044], 0.9750, 0.9530, 0.9358, 0.9099, 0.8729, 0.9524, Acc_f: 0.1044, Acc_r: 0.9437
[train] epoch 3, batch 14, loss 2.5161426067352295
0.9765, 0.9747, 0.9397, [0.0527], 0.9750, 0.9551, 0.9358, 0.9104, 0.8687, 0.9448, Acc_f: 0.0527, Acc_r: 0.9423
[train] epoch 4, batch 14, loss 2.4118316173553467
0.9759, 0.9749, 0.9397, [0.0288], 0.9750, 0.9572, 0.9358, 0.9094, 0.8645, 0.9404, Acc_f: 0.0288, Acc_r: 0.9414
[train] epoch 5, batch 14, loss 2.4532108306884766
0.9765, 0.9747, 0.9400, [0.0167

In [27]:
Acc_r = 100*np.array([0.9405, 0.9424, 0.9264])
Acc_f = 100*np.array([0.0167, 0.0593, 0.0128])

print(f'Random label + Subspace Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'Random label + Subspace Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

Random label + Subspace Acc_f: 2.96 \pm 2.11
Random label + Subspace Acc_r: 93.64 \pm 0.71


In [18]:
def get_pseudo_label(args, model, x):
    masked_output = model(x)
    masked_output[:, args.unlearn_class] = -np.inf
    pseudo_labels = torch.topk(masked_output, k=1, dim=1).indices
    return pseudo_labels.reshape(-1)

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.005)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(15):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = get_pseudo_label(args, model, x)
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9719, 0.9712, 0.9624, [0.9167], 0.9687, 0.9488, 0.9474, 0.9534, 0.9253, 0.9611, Acc_f: 0.9167, Acc_r: 0.9567
[train] epoch 0, batch 14, loss 0.8711209297180176
0.9713, 0.9792, 0.9706, [0.0160], 0.9620, 0.9685, 0.9353, 0.9227, 0.9048, 0.9618, Acc_f: 0.0160, Acc_r: 0.9529
[train] epoch 1, batch 14, loss 0.5616824626922607
0.9719, 0.9800, 0.9723, [0.0024], 0.9600, 0.9669, 0.9393, 0.9257, 0.9193, 0.9586, Acc_f: 0.0024, Acc_r: 0.9549
[train] epoch 2, batch 14, loss 0.47658926248550415
0.9713, 0.9802, 0.9718, [0.0007], 0.9596, 0.9664, 0.9423, 0.9257, 0.9265, 0.9580, Acc_f: 0.0007, Acc_r: 0.9558
[train] epoch 3, batch 14, loss 0.3886551558971405
0.9713, 0.9804, 0.9723, [0.0000], 0.9580, 0.9656, 0.9439, 0.9277, 0.9307, 0.9580, Acc_f: 0.0000, Acc_r: 0.9564
[train] epoch 4, batch 14, loss 0.4081674814224243
0.9719, 0.9804, 0.9723, [0.0000], 0.9576, 0.9648, 0.9423, 0.9297, 0.9337, 0.9580, Acc_f: 0.0000, Acc_r: 0.9567
[train] epoch 5, batch 14, loss 0.40309152007102966
0.9719, 0.9823, 0.9689, [0

In [2]:
Acc_r = 100*np.array([0.9590, 0.9550, 0.9583])
Acc_f = 100*np.array([0.0000, 0.0000, 0.0000])

print(f'UNSC Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'UNSC Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

UNSC Acc_f: 0.00 \pm 0.00
UNSC Acc_r: 95.74 \pm 0.17


In [1]:
import numpy as np
a = (100*np.array([0.9590, 0.9550, 0.9583])-100*np.array([0.9567, 0.9539, 0.9549]))
a.mean(), a.std()

(0.2266666666666642, 0.09392668535737063)

In [21]:
Acc_r, Acc_f = np.zeros(3), np.zeros(3)
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.0001)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d):
            m.eval()

    for ep in range(10):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = torch.from_numpy(np.random.choice(remain_class, size=x.shape[0])).cuda()
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        Acc_r[i], Acc_f[i] = test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9719, 0.9712, 0.9624, [0.9167], 0.9687, 0.9488, 0.9474, 0.9534, 0.9253, 0.9611, Acc_f: 0.9167, Acc_r: 0.9567
[train] epoch 0, batch 14, loss 5.851011276245117
0.9731, 0.9774, 0.9605, [0.8543], 0.9671, 0.9534, 0.9433, 0.9326, 0.9187, 0.9599, Acc_f: 0.8543, Acc_r: 0.9540
[train] epoch 1, batch 14, loss 3.5684101581573486
0.9725, 0.9847, 0.9369, [0.6020], 0.9560, 0.9442, 0.9160, 0.8534, 0.8602, 0.9423, Acc_f: 0.6020, Acc_r: 0.9296
[train] epoch 2, batch 14, loss 2.9119174480438232
0.9685, 0.9867, 0.8802, [0.2623], 0.9493, 0.9023, 0.8680, 0.7504, 0.7765, 0.8489, Acc_f: 0.2623, Acc_r: 0.8812
[train] epoch 3, batch 14, loss 2.694432258605957
0.9644, 0.9874, 0.8260, [0.1329], 0.9465, 0.8595, 0.8235, 0.6667, 0.7078, 0.7687, Acc_f: 0.1329, Acc_r: 0.8389
[train] epoch 4, batch 14, loss 2.5529770851135254
0.9599, 0.9880, 0.7848, [0.0697], 0.9429, 0.8305, 0.7931, 0.6147, 0.6560, 0.7053, Acc_f: 0.0697, Acc_r: 0.8084
[train] epoch 5, batch 14, loss 2.6163973808288574
0.9581, 0.9878, 0.7549, [0.042

In [22]:
Acc_f = 100*np.array([0.0180, 0.0257, 0.0087])
Acc_r = 100*np.array([0.7548, 0.8490, 0.7047])
print(f'Random label Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'Random label Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

Random label Acc_f: 1.75 \pm 0.70
Random label Acc_r: 76.95 \pm 5.98


# Boundary Unlearning

In [33]:
from agents.adv import FGSM

def find_adjacent_cls(adv_agent, x, y):
    x_adv = adv_agent.perturb(x, y)
    adv_logits = model(x_adv)
    adv_pred = torch.argmax(adv_logits.data, 1)
    return adv_pred, x_adv

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    adv_agent = FGSM(deepcopy(model), bound=0.5, norm=False, random_start=True, device='cuda')
    sgd_mr_model = deepcopy(model)
    print('==='*20)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.0005)

    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()
            
    model.eval()
    for ep in range(20):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            adv_pred, x_adv = find_adjacent_cls(adv_agent, x, y)
            adv_y = torch.argmax(model(x_adv), dim=1).detach().cuda()
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, adv_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9719, 0.9712, 0.9624, [0.9167], 0.9687, 0.9488, 0.9474, 0.9534, 0.9253, 0.9611, Acc_f: 0.9167, Acc_r: 0.9567
[train] epoch 0, batch 14, loss 2.40285062789917
0.9553, 0.9922, 0.8265, [0.1513], 0.9291, 0.8775, 0.8270, 0.6568, 0.7596, 0.7592, Acc_f: 0.1513, Acc_r: 0.8426
[train] epoch 1, batch 14, loss 2.193714141845703
0.9404, 0.9941, 0.7619, [0.0545], 0.9005, 0.8658, 0.7618, 0.5597, 0.7054, 0.5774, Acc_f: 0.0545, Acc_r: 0.7852
[train] epoch 2, batch 14, loss 2.172280788421631
0.9278, 0.9945, 0.7293, [0.0295], 0.8720, 0.8842, 0.7294, 0.5017, 0.6831, 0.4803, Acc_f: 0.0295, Acc_r: 0.7558
[train] epoch 3, batch 14, loss 2.1772871017456055
0.9163, 0.9949, 0.6802, [0.0142], 0.8450, 0.8893, 0.6970, 0.4522, 0.6645, 0.3737, Acc_f: 0.0142, Acc_r: 0.7237
[train] epoch 4, batch 14, loss 2.1005611419677734
0.9123, 0.9953, 0.6385, [0.0097], 0.8161, 0.8859, 0.6763, 0.4185, 0.6542, 0.3122, Acc_f: 0.0097, Acc_r: 0.7010
[train] epoch 5, batch 14, loss 2.0606777667999268
0.9083, 0.9953, 0.6247, [0.0073]

# Unlearn 2 classes

In [34]:
num_unlearn = 2
arxiv_name = dict[num_unlearn]['arxiv_name']
args.unlearn_class = dict[num_unlearn]['unlean_class']
Acc_r, Acc_f = np.zeros((2,3)), np.zeros((2,3))
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    Acc_f[0][i], Acc_r[0][i] = test_by_class(model, test_loader, i=args.unlearn_class)

print('------------ Retrained model ------------')
for i in range(3):
    model_r = get_model(args)
    try:
        model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/retrain_model_{arxiv_name}_{args.seeds[i]}.pth'))
        Acc_f[1][i], Acc_r[1][i] = test_by_class(model_r, test_loader, i=args.unlearn_class)
    except:
        print('No retrained model')
        break

print(f'Original model Acc_f: {100*Acc_f[0].mean():.2f} \pm {100*Acc_f[0].std():.2f}')
print(f'Original model Acc_r: {100*Acc_r[0].mean():.2f} \pm {100*Acc_r[0].std():.2f}')

print(f'Retrained model Acc_f: {100*Acc_f[1].mean():.2f} \pm {100*Acc_f[1].std():.2f}')
print(f'Retrained model Acc_r: {100*Acc_r[1].mean():.2f} \pm {100*Acc_r[1].std():.2f}')


0.9719, [0.9712], 0.9624, [0.9167], 0.9687, 0.9488, 0.9474, 0.9534, 0.9253, 0.9611, Acc_f: 0.9439, Acc_r: 0.9549
0.9639, [0.9722], 0.9658, [0.9129], 0.9691, 0.9480, 0.9646, 0.9287, 0.9253, 0.9473, Acc_f: 0.9425, Acc_r: 0.9516
0.9616, [0.9645], 0.9655, [0.9094], 0.9734, 0.9534, 0.9560, 0.9376, 0.9361, 0.9461, Acc_f: 0.9370, Acc_r: 0.9537
------------ Retrained model ------------
0.9622, [0.0000], 0.9730, [0.0000], 0.9766, 0.9606, 0.9560, 0.9797, 0.9428, 0.9561, Acc_f: 0.0000, Acc_r: 0.9634
0.9702, [0.0000], 0.9718, [0.0000], 0.9723, 0.9543, 0.9600, 0.9777, 0.9476, 0.9592, Acc_f: 0.0000, Acc_r: 0.9641
0.9644, [0.0000], 0.9747, [0.0000], 0.9786, 0.9551, 0.9504, 0.9792, 0.9416, 0.9542, Acc_f: 0.0000, Acc_r: 0.9623
Original model Acc_f: 94.11 \pm 0.30
Original model Acc_r: 95.34 \pm 0.14
Retrained model Acc_f: 0.00 \pm 0.00
Retrained model Acc_r: 96.33 \pm 0.08


In [35]:
train_loader, val_loader, test_loader = get_dataloader(args)
remain_train_loader, unlearn_train_loader = split_2_remain_unlearn(args, train_loader)
remain_val_loader, unlearn_val_loader = split_2_remain_unlearn(args, val_loader)
remain_test_loader, unlearn_test_loader = split_2_remain_unlearn(args, test_loader)
remain_class = np.setdiff1d(np.arange(args.num_classes), args.unlearn_class)

Proj_mat_lst =[]
train_targets_list = np.array(train_loader.dataset.labels)[train_loader.sampler.indices]
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    print(f'------------ Trail {i} ------------')
    merged_feat_mat = []
    for cls_id in range(10): 
        cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
        cls_indices = train_loader.sampler.indices[cls_indices]
        cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
        cls_loader_dict = torch.utils.data.DataLoader(train_loader.dataset, 
                                                                batch_size=args.batch_size, 
                                                                sampler=cls_sampler)
        if cls_id in args.unlearn_class:
            continue
        for batch, (x, y) in enumerate(cls_loader_dict ):
            x = x.cuda()
            y = y.cuda()
            mat_list = get_representation_matrix(model, 
                                                x, 
                                                batch_list=[24, 100, 100, 100, 125, 125, 125, 250, 250, 256, 256])
            break
        threshold = 0.97 + 0.003*cls_id
        merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
        proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]
        Proj_mat_lst.append(proj_mat)

Using downloaded and verified file: ../data/svhn/train_32x32.mat
Using downloaded and verified file: ../data/svhn/test_32x32.mat
------------ Trail 0 ------------
Threshold:  0.97
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 4/27
Layer 2 : 31/576
Layer 3 : 409/1152
Layer 4 : 1087/2304
Layer 5 : 364/2304
Layer 6 : 362/4608
Layer 7 : 0/4608
Layer 8 : 0/4608
Layer 9 : 3/512
Layer 10 : 0/4096
Layer 11 : 0/4096
----------------------------------------
Threshold:  0.976
Skip Updating GPM for layer: 7
Skip Updating GPM for layer: 8
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/27
Layer 2 : 40/576
Layer 3 : 586/1152
Layer 4 : 1630/2304
Layer 5 : 734/2304
Layer 6 : 754/4608
Layer 7 : 0/4608
Layer 8 : 0/4608
Layer 9 : 10/512
Layer 10 : 4/4096
Layer 11 : 2/4096
----------------------------------------
Threshold:  0.982
Skip Updating GPM for layer

In [36]:
def get_pseudo_label(args, model, x):
    masked_output = model(x)
    masked_output[:, args.unlearn_class] = -np.inf
    pseudo_labels = torch.topk(masked_output, k=1, dim=1).indices
    return pseudo_labels.reshape(-1)

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.005)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(15):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = get_pseudo_label(args, model, x)
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9719, [0.9712], 0.9624, [0.9167], 0.9687, 0.9488, 0.9474, 0.9534, 0.9253, 0.9611, Acc_f: 0.9439, Acc_r: 0.9549
[train] epoch 0, batch 39, loss 0.5830514430999756
0.9794, [0.0002], 0.9764, [0.0056], 0.9790, 0.9643, 0.9342, 0.9871, 0.8928, 0.9549, Acc_f: 0.0029, Acc_r: 0.9585
[train] epoch 1, batch 39, loss 0.3588450849056244
0.9799, [0.0000], 0.9769, [0.0003], 0.9806, 0.9660, 0.9418, 0.9856, 0.9223, 0.9530, Acc_f: 0.0002, Acc_r: 0.9633
[train] epoch 2, batch 39, loss 0.3573075234889984
0.9794, [0.0000], 0.9771, [0.0003], 0.9806, 0.9664, 0.9413, 0.9856, 0.9337, 0.9524, Acc_f: 0.0002, Acc_r: 0.9646
[train] epoch 3, batch 39, loss 0.34689757227897644
0.9794, [0.0000], 0.9759, [0.0000], 0.9814, 0.9669, 0.9393, 0.9842, 0.9367, 0.9542, Acc_f: 0.0000, Acc_r: 0.9647
[train] epoch 4, batch 39, loss 0.2947760820388794
0.9805, [0.0000], 0.9732, [0.0000], 0.9814, 0.9664, 0.9383, 0.9861, 0.9373, 0.9549, Acc_f: 0.0000, Acc_r: 0.9648
[train] epoch 5, batch 39, loss 0.2616407573223114
0.9799, [0.0000

In [4]:
Acc_r = 100*np.array([0.9648, 0.9624, 0.9659])
Acc_f = 100*np.array([0.0000, 0.0000, 0.0000])

print(f'UNSC Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'UNSC Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

UNSC Acc_f: 0.00 \pm 0.00
UNSC Acc_r: 96.44 \pm 0.15


In [3]:
a = (100*np.array([0.9648, 0.9624, 0.9659])-100*np.array([0.9549, 0.9516, 0.9537]))
a.mean(), a.std()

(1.0966666666666736, 0.09463379711051795)

# Unlearn 3 classes

In [39]:
num_unlearn = 3
arxiv_name = dict[num_unlearn]['arxiv_name']
args.unlearn_class = dict[num_unlearn]['unlean_class']
Acc_r, Acc_f = np.zeros((2,3)), np.zeros((2,3))
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    Acc_f[0][i], Acc_r[0][i] = test_by_class(model, test_loader, i=args.unlearn_class)

print('------------ Retrained model ------------')
for i in range(3):
    model_r = get_model(args)
    try:
        model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/retrain_model_{arxiv_name}_{args.seeds[i]}.pth'))
        Acc_f[1][i], Acc_r[1][i] = test_by_class(model_r, test_loader, i=args.unlearn_class)
    except:
        print('No retrained model')
        break

print(f'Original model Acc_f: {100*Acc_f[0].mean():.2f} \pm {100*Acc_f[0].std():.2f}')
print(f'Original model Acc_r: {100*Acc_r[0].mean():.2f} \pm {100*Acc_r[0].std():.2f}')

print(f'Retrained model Acc_f: {100*Acc_f[1].mean():.2f} \pm {100*Acc_f[1].std():.2f}')
print(f'Retrained model Acc_r: {100*Acc_r[1].mean():.2f} \pm {100*Acc_r[1].std():.2f}')


0.9719, [0.9712], 0.9624, [0.9167], 0.9687, [0.9488], 0.9474, 0.9534, 0.9253, 0.9611, Acc_f: 0.9456, Acc_r: 0.9558
0.9639, [0.9722], 0.9658, [0.9129], 0.9691, [0.9480], 0.9646, 0.9287, 0.9253, 0.9473, Acc_f: 0.9443, Acc_r: 0.9521
0.9616, [0.9645], 0.9655, [0.9094], 0.9734, [0.9534], 0.9560, 0.9376, 0.9361, 0.9461, Acc_f: 0.9425, Acc_r: 0.9538
------------ Retrained model ------------
0.9748, [0.0000], 0.9723, [0.0000], 0.9754, [0.0000], 0.9661, 0.9767, 0.9488, 0.9517, Acc_f: 0.0000, Acc_r: 0.9665
0.9335, [0.0000], 0.9689, [0.0000], 0.9643, [0.0000], 0.9555, 0.9470, 0.9301, 0.9003, Acc_f: 0.0000, Acc_r: 0.9428
0.9656, [0.0000], 0.9785, [0.0000], 0.9746, [0.0000], 0.9727, 0.9757, 0.9452, 0.9611, Acc_f: 0.0000, Acc_r: 0.9676
Original model Acc_f: 94.41 \pm 0.13
Original model Acc_r: 95.39 \pm 0.15
Retrained model Acc_f: 0.00 \pm 0.00
Retrained model Acc_r: 95.90 \pm 1.15


In [40]:
train_loader, val_loader, test_loader = get_dataloader(args)
remain_train_loader, unlearn_train_loader = split_2_remain_unlearn(args, train_loader)
remain_val_loader, unlearn_val_loader = split_2_remain_unlearn(args, val_loader)
remain_test_loader, unlearn_test_loader = split_2_remain_unlearn(args, test_loader)
remain_class = np.setdiff1d(np.arange(args.num_classes), args.unlearn_class)

Proj_mat_lst =[]
train_targets_list = np.array(train_loader.dataset.labels)[train_loader.sampler.indices]
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    print(f'------------ Trail {i} ------------')
    merged_feat_mat = []
    for cls_id in range(10): 
        cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
        cls_indices = train_loader.sampler.indices[cls_indices]
        cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
        cls_loader_dict = torch.utils.data.DataLoader(train_loader.dataset, 
                                                                batch_size=args.batch_size, 
                                                                sampler=cls_sampler)
        if cls_id in args.unlearn_class:
            continue
        for batch, (x, y) in enumerate(cls_loader_dict ):
            x = x.cuda()
            y = y.cuda()
            mat_list = get_representation_matrix(model, 
                                                x, 
                                                batch_list=[24, 100, 100, 100, 125, 125, 125, 250, 250, 256, 256])
            break
        threshold = 0.97 + 0.003*cls_id
        merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
        proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]
        Proj_mat_lst.append(proj_mat)

Using downloaded and verified file: ../data/svhn/train_32x32.mat
Using downloaded and verified file: ../data/svhn/test_32x32.mat
------------ Trail 0 ------------
Threshold:  0.97
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 4/27
Layer 2 : 31/576
Layer 3 : 425/1152
Layer 4 : 1085/2304
Layer 5 : 361/2304
Layer 6 : 360/4608
Layer 7 : 0/4608
Layer 8 : 0/4608
Layer 9 : 2/512
Layer 10 : 0/4096
Layer 11 : 0/4096
----------------------------------------
Threshold:  0.976
Skip Updating GPM for layer: 7
Skip Updating GPM for layer: 8
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/27
Layer 2 : 38/576
Layer 3 : 576/1152
Layer 4 : 1609/2304
Layer 5 : 719/2304
Layer 6 : 735/4608
Layer 7 : 0/4608
Layer 8 : 0/4608
Layer 9 : 9/512
Layer 10 : 4/4096
Layer 11 : 2/4096
----------------------------------------
Threshold:  0.982
Skip Updating GPM for layer:

In [41]:
def get_pseudo_label(args, model, x):
    masked_output = model(x)
    masked_output[:, args.unlearn_class] = -np.inf
    pseudo_labels = torch.topk(masked_output, k=1, dim=1).indices
    return pseudo_labels.reshape(-1)

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.005)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(15):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = get_pseudo_label(args, model, x)
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9719, [0.9712], 0.9624, [0.9167], 0.9687, [0.9488], 0.9474, 0.9534, 0.9253, 0.9611, Acc_f: 0.9456, Acc_r: 0.9558
[train] epoch 0, batch 51, loss 0.5003509521484375
0.9799, [0.0000], 0.9802, [0.0000], 0.9794, [0.0055], 0.9565, 0.9891, 0.8849, 0.9555, Acc_f: 0.0018, Acc_r: 0.9608
[train] epoch 1, batch 51, loss 0.3797537684440613
0.9805, [0.0000], 0.9798, [0.0000], 0.9822, [0.0013], 0.9575, 0.9881, 0.9307, 0.9555, Acc_f: 0.0004, Acc_r: 0.9678
[train] epoch 2, batch 51, loss 0.35247498750686646
0.9811, [0.0000], 0.9812, [0.0000], 0.9826, [0.0004], 0.9575, 0.9856, 0.9337, 0.9561, Acc_f: 0.0001, Acc_r: 0.9683
[train] epoch 3, batch 51, loss 0.30832216143608093
0.9782, [0.0000], 0.9800, [0.0000], 0.9822, [0.0004], 0.9600, 0.9866, 0.9349, 0.9561, Acc_f: 0.0001, Acc_r: 0.9683
[train] epoch 4, batch 51, loss 0.3369174897670746
0.9805, [0.0000], 0.9812, [0.0000], 0.9822, [0.0004], 0.9590, 0.9866, 0.9325, 0.9555, Acc_f: 0.0001, Acc_r: 0.9682
[train] epoch 5, batch 51, loss 0.2328011691570282
0.

In [6]:
Acc_r = 100*np.array([0.9683, 0.9651, 0.9688])
Acc_f = 100*np.array([0.0004, 0.0000, 0.0000])

print(f'UNSC Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'UNSC Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

UNSC Acc_f: 0.01 \pm 0.02
UNSC Acc_r: 96.74 \pm 0.16


# Unlearn 4 classes

In [13]:
num_unlearn = 4
arxiv_name = dict[num_unlearn]['arxiv_name']
args.unlearn_class = dict[num_unlearn]['unlean_class']
Acc_r, Acc_f = np.zeros((2,3)), np.zeros((2,3))
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    Acc_f[0][i], Acc_r[0][i] = test_by_class(model, test_loader, i=args.unlearn_class)

print('------------ Retrained model ------------')
for i in range(3):
    model_r = get_model(args)
    try:
        model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/retrain_model_{arxiv_name}_{args.seeds[i]}.pth'))
        Acc_f[1][i], Acc_r[1][i] = test_by_class(model_r, test_loader, i=args.unlearn_class)
    except:
        print('No retrained model')
        break

print(f'Original model Acc_f: {100*Acc_f[0].mean():.2f} \pm {100*Acc_f[0].std():.2f}')
print(f'Original model Acc_r: {100*Acc_r[0].mean():.2f} \pm {100*Acc_r[0].std():.2f}')

print(f'Retrained model Acc_f: {100*Acc_f[1].mean():.2f} \pm {100*Acc_f[1].std():.2f}')
print(f'Retrained model Acc_r: {100*Acc_r[1].mean():.2f} \pm {100*Acc_r[1].std():.2f}')


0.9719, [0.9712], 0.9624, [0.9167], 0.9687, [0.9488], 0.9474, 0.9534, [0.9253], 0.9611, Acc_f: 0.9405, Acc_r: 0.9608
0.9639, [0.9722], 0.9658, [0.9129], 0.9691, [0.9480], 0.9646, 0.9287, [0.9253], 0.9473, Acc_f: 0.9396, Acc_r: 0.9566
0.9616, [0.9645], 0.9655, [0.9094], 0.9734, [0.9534], 0.9560, 0.9376, [0.9361], 0.9461, Acc_f: 0.9409, Acc_r: 0.9567
------------ Retrained model ------------
0.9581, [0.0000], 0.9771, [0.0000], 0.9746, [0.0000], 0.9722, 0.9807, [0.0000], 0.9592, Acc_f: 0.0000, Acc_r: 0.9703
0.9639, [0.0000], 0.9761, [0.0000], 0.9810, [0.0000], 0.9712, 0.9782, [0.0000], 0.9567, Acc_f: 0.0000, Acc_r: 0.9712
0.9610, [0.0000], 0.9742, [0.0000], 0.9818, [0.0000], 0.9757, 0.9822, [0.0000], 0.9342, Acc_f: 0.0000, Acc_r: 0.9682
Original model Acc_f: 94.03 \pm 0.05
Original model Acc_r: 95.80 \pm 0.20
Retrained model Acc_f: 0.00 \pm 0.00
Retrained model Acc_r: 96.99 \pm 0.13


In [44]:
train_loader, val_loader, test_loader = get_dataloader(args)
remain_train_loader, unlearn_train_loader = split_2_remain_unlearn(args, train_loader)
remain_val_loader, unlearn_val_loader = split_2_remain_unlearn(args, val_loader)
remain_test_loader, unlearn_test_loader = split_2_remain_unlearn(args, test_loader)
remain_class = np.setdiff1d(np.arange(args.num_classes), args.unlearn_class)

Proj_mat_lst =[]
train_targets_list = np.array(train_loader.dataset.labels)[train_loader.sampler.indices]
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    print(f'------------ Trail {i} ------------')
    merged_feat_mat = []
    for cls_id in range(10): 
        cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
        cls_indices = train_loader.sampler.indices[cls_indices]
        cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
        cls_loader_dict = torch.utils.data.DataLoader(train_loader.dataset, 
                                                                batch_size=args.batch_size, 
                                                                sampler=cls_sampler)
        if cls_id in args.unlearn_class:
            continue
        for batch, (x, y) in enumerate(cls_loader_dict ):
            x = x.cuda()
            y = y.cuda()
            mat_list = get_representation_matrix(model, 
                                                x, 
                                                batch_list=[24, 100, 100, 100, 125, 125, 125, 250, 250, 256, 256])
            break
        threshold = 0.97 + 0.003*cls_id
        merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
        proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]
        Proj_mat_lst.append(proj_mat)

Using downloaded and verified file: ../data/svhn/train_32x32.mat
Using downloaded and verified file: ../data/svhn/test_32x32.mat
------------ Trail 0 ------------
Threshold:  0.97
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 4/27
Layer 2 : 30/576
Layer 3 : 407/1152
Layer 4 : 1083/2304
Layer 5 : 365/2304
Layer 6 : 371/4608
Layer 7 : 0/4608
Layer 8 : 0/4608
Layer 9 : 3/512
Layer 10 : 0/4096
Layer 11 : 0/4096
----------------------------------------
Threshold:  0.976
Skip Updating GPM for layer: 7
Skip Updating GPM for layer: 8
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/27
Layer 2 : 39/576
Layer 3 : 583/1152
Layer 4 : 1624/2304
Layer 5 : 732/2304
Layer 6 : 755/4608
Layer 7 : 0/4608
Layer 8 : 0/4608
Layer 9 : 10/512
Layer 10 : 4/4096
Layer 11 : 2/4096
----------------------------------------
Threshold:  0.982
Skip Updating GPM for layer

In [45]:
def get_pseudo_label(args, model, x):
    masked_output = model(x)
    masked_output[:, args.unlearn_class] = -np.inf
    pseudo_labels = torch.topk(masked_output, k=1, dim=1).indices
    return pseudo_labels.reshape(-1)

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.005)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(15):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = get_pseudo_label(args, model, x)
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9719, [0.9712], 0.9624, [0.9167], 0.9687, [0.9488], 0.9474, 0.9534, [0.9253], 0.9611, Acc_f: 0.9405, Acc_r: 0.9608
[train] epoch 0, batch 60, loss 0.6527426242828369
0.9771, [0.0008], 0.9798, [0.0000], 0.9806, [0.0042], 0.9626, 0.9881, [0.0006], 0.9599, Acc_f: 0.0014, Acc_r: 0.9747
[train] epoch 1, batch 60, loss 0.3771827816963196
0.9725, [0.0000], 0.9817, [0.0000], 0.9834, [0.0013], 0.9626, 0.9866, [0.0000], 0.9643, Acc_f: 0.0003, Acc_r: 0.9752
[train] epoch 2, batch 60, loss 0.26301056146621704
0.9765, [0.0000], 0.9824, [0.0000], 0.9814, [0.0000], 0.9666, 0.9866, [0.0000], 0.9636, Acc_f: 0.0000, Acc_r: 0.9762
[train] epoch 3, batch 60, loss 0.32000425457954407
0.9759, [0.0000], 0.9798, [0.0000], 0.9830, [0.0004], 0.9646, 0.9861, [0.0000], 0.9649, Acc_f: 0.0001, Acc_r: 0.9757
[train] epoch 4, batch 60, loss 0.29089492559432983
0.9748, [0.0000], 0.9822, [0.0000], 0.9830, [0.0004], 0.9681, 0.9876, [0.0000], 0.9624, Acc_f: 0.0001, Acc_r: 0.9763
[train] epoch 5, batch 60, loss 0.186358

In [ ]:
Acc_r = 100*np.array([0.9763, 0.9739, 0.9750])
Acc_f = 100*np.array([0.0001, 0.0000, 0.0002])

print(f'UNSC Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'UNSC Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

Acc_r = 100*np.array([0.9608, 0.9566, 0.9567])
Acc_f = 100*np.array([0.9405, 0.9396, 0.9409])
print(f'Origin Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'Origin Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

In [5]:
a = (100*np.array([0.9763, 0.9739, 0.9750])-100*np.array([0.9608, 0.95661, 0.9567]))
a.mean(), a.std()

(1.7029999999999983, 0.11577852420318199)

# Unlearn 5 classes

In [47]:
num_unlearn = 5
arxiv_name = dict[num_unlearn]['arxiv_name']
args.unlearn_class = dict[num_unlearn]['unlean_class']
Acc_r, Acc_f = np.zeros((2,3)), np.zeros((2,3))
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    Acc_f[0][i], Acc_r[0][i] = test_by_class(model, test_loader, i=args.unlearn_class)

print('------------ Retrained model ------------')
for i in range(3):
    model_r = get_model(args)
    try:
        model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/retrain_model_{arxiv_name}_{args.seeds[i]}.pth'))
        Acc_f[1][i], Acc_r[1][i] = test_by_class(model_r, test_loader, i=args.unlearn_class)
    except:
        print('No retrained model')
        break

print(f'Original model Acc_f: {100*Acc_f[0].mean():.2f} \pm {100*Acc_f[0].std():.2f}')
print(f'Original model Acc_r: {100*Acc_r[0].mean():.2f} \pm {100*Acc_r[0].std():.2f}')

print(f'Retrained model Acc_f: {100*Acc_f[1].mean():.2f} \pm {100*Acc_f[1].std():.2f}')
print(f'Retrained model Acc_r: {100*Acc_r[1].mean():.2f} \pm {100*Acc_r[1].std():.2f}')


0.9719, [0.9712], 0.9624, 0.9167, 0.9687, [0.9488], [0.9474], [0.9534], 0.9253, [0.9611], Acc_f: 0.9564, Acc_r: 0.9490
0.9639, [0.9722], 0.9658, 0.9129, 0.9691, [0.9480], [0.9646], [0.9287], 0.9253, [0.9473], Acc_f: 0.9521, Acc_r: 0.9474
0.9616, [0.9645], 0.9655, 0.9094, 0.9734, [0.9534], [0.9560], [0.9376], 0.9361, [0.9461], Acc_f: 0.9515, Acc_r: 0.9492
------------ Retrained model ------------
0.9811, [0.0000], 0.9764, 0.9473, 0.9711, [0.0000], [0.0000], [0.0000], 0.9663, [0.0000], Acc_f: 0.0000, Acc_r: 0.9684
0.9748, [0.0000], 0.9728, 0.9480, 0.9754, [0.0000], [0.0000], [0.0000], 0.9560, [0.0000], Acc_f: 0.0000, Acc_r: 0.9654
0.9719, [0.0000], 0.9706, 0.9410, 0.9727, [0.0000], [0.0000], [0.0000], 0.9663, [0.0000], Acc_f: 0.0000, Acc_r: 0.9645
Original model Acc_f: 95.34 \pm 0.22
Original model Acc_r: 94.85 \pm 0.08
Retrained model Acc_f: 0.00 \pm 0.00
Retrained model Acc_r: 96.61 \pm 0.17


In [48]:
train_loader, val_loader, test_loader = get_dataloader(args)
remain_train_loader, unlearn_train_loader = split_2_remain_unlearn(args, train_loader)
remain_val_loader, unlearn_val_loader = split_2_remain_unlearn(args, val_loader)
remain_test_loader, unlearn_test_loader = split_2_remain_unlearn(args, test_loader)
remain_class = np.setdiff1d(np.arange(args.num_classes), args.unlearn_class)

Proj_mat_lst =[]
train_targets_list = np.array(train_loader.dataset.labels)[train_loader.sampler.indices]
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    print(f'------------ Trail {i} ------------')
    merged_feat_mat = []
    for cls_id in range(10): 
        cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
        cls_indices = train_loader.sampler.indices[cls_indices]
        cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
        cls_loader_dict = torch.utils.data.DataLoader(train_loader.dataset, 
                                                                batch_size=args.batch_size, 
                                                                sampler=cls_sampler)
        if cls_id in args.unlearn_class:
            continue
        for batch, (x, y) in enumerate(cls_loader_dict ):
            x = x.cuda()
            y = y.cuda()
            mat_list = get_representation_matrix(model, 
                                                x, 
                                                batch_list=[24, 100, 100, 100, 125, 125, 125, 250, 250, 256, 256])
            break
        threshold = 0.97 + 0.003*cls_id
        merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
        proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]
        Proj_mat_lst.append(proj_mat)

Using downloaded and verified file: ../data/svhn/train_32x32.mat
Using downloaded and verified file: ../data/svhn/test_32x32.mat
------------ Trail 0 ------------
Threshold:  0.97
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 4/27
Layer 2 : 31/576
Layer 3 : 415/1152
Layer 4 : 1066/2304
Layer 5 : 355/2304
Layer 6 : 354/4608
Layer 7 : 0/4608
Layer 8 : 0/4608
Layer 9 : 2/512
Layer 10 : 0/4096
Layer 11 : 0/4096
----------------------------------------
Threshold:  0.976
Skip Updating GPM for layer: 7
Skip Updating GPM for layer: 8
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 6/27
Layer 2 : 39/576
Layer 3 : 582/1152
Layer 4 : 1618/2304
Layer 5 : 719/2304
Layer 6 : 735/4608
Layer 7 : 0/4608
Layer 8 : 0/4608
Layer 9 : 8/512
Layer 10 : 3/4096
Layer 11 : 2/4096
----------------------------------------
Threshold:  0.979
Skip Updating GPM for layer:

In [49]:
def get_pseudo_label(args, model, x):
    masked_output = model(x)
    masked_output[:, args.unlearn_class] = -np.inf
    pseudo_labels = torch.topk(masked_output, k=1, dim=1).indices
    return pseudo_labels.reshape(-1)

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.005)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(15):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = get_pseudo_label(args, model, x)
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9719, [0.9712], 0.9624, 0.9167, 0.9687, [0.9488], [0.9474], [0.9534], 0.9253, [0.9611], Acc_f: 0.9564, Acc_r: 0.9490
[train] epoch 0, batch 64, loss 0.38560330867767334
0.9868, [0.0002], 0.9870, 0.9587, 0.9734, [0.0025], [0.0020], [0.0000], 0.9554, [0.0063], Acc_f: 0.0022, Acc_r: 0.9723
[train] epoch 1, batch 64, loss 0.30492883920669556
0.9862, [0.0000], 0.9846, 0.9639, 0.9798, [0.0008], [0.0000], [0.0000], 0.9566, [0.0006], Acc_f: 0.0003, Acc_r: 0.9742
[train] epoch 2, batch 64, loss 0.2505352795124054
0.9851, [0.0000], 0.9858, 0.9611, 0.9802, [0.0004], [0.0000], [0.0000], 0.9584, [0.0000], Acc_f: 0.0001, Acc_r: 0.9741
[train] epoch 3, batch 64, loss 0.18862971663475037
0.9857, [0.0000], 0.9851, 0.9615, 0.9794, [0.0000], [0.0000], [0.0000], 0.9578, [0.0000], Acc_f: 0.0000, Acc_r: 0.9739
[train] epoch 4, batch 64, loss 0.223812535405159
0.9857, [0.0000], 0.9834, 0.9646, 0.9802, [0.0000], [0.0000], [0.0000], 0.9584, [0.0000], Acc_f: 0.0000, Acc_r: 0.9745
[train] epoch 5, batch 64, lo

In [ ]:
Acc_r = 100*np.array([0.9745, 0.9754, 0.9744])
Acc_f = 100*np.array([0.0000, 0.0000, 0.0001])
print(f'UNSC Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'UNSC Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

Acc_r = 100*np.array([0.9490, 0.9474, 0.9492])
Acc_f = 100*np.array([0.9564, 0.9521, 0.9515])
print(f'Origin Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'Origin Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')